In [196]:
from preprocessing import Make_Dataset
import numpy as np
import pandas as pd
from tqdm import tqdm
import gc
pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment',  None)

In [2]:
data=Make_Dataset.Integration_data()

Total stay 14474


100%|██████████| 14474/14474 [02:08<00:00, 112.69it/s]


total stay dataframe shape (2543197, 168)


TypeError: __init__() should return None, not 'DataFrame'

In [197]:
data=pd.read_csv('Total.csv', index_col=0)
data = data.reset_index()
data['index'] = (data['index'].values)
data = data.rename(columns = {'index': 'Time_since_ICU_admission'})

In [198]:
data = data.rename(columns = {'Unnamed: 161_level_1':'height', 'Unnamed: 162_level_1':'weight'})
data['weight_fillna'] = 0
data['height_fillna'] = 0
idx_w = data[data['weight']==0].index
idx_h = data[data['height']==0].index

data['weight_fillna'].loc[idx_w] = 1
data['height_fillna'].loc[idx_h] = 1

In [199]:
data['Temperature'] = data['Temperature.1']

idx = data[(data['PEEP'].isnull()) & ~(data['PEEP.1'].isnull())].index
data['PEEP'].loc[idx] = data['PEEP.1'].loc[idx].values

idx = data[~(data['FIO2 (%)'].isnull()) & (data['FIO2 (%).1'].isnull())].index
data['FIO2 (%).1'].loc[idx] = data['FIO2 (%)'].loc[idx].values

idx = data[(data['O2 Sat (%)'].isnull()) & ~(data['O2 Sat (%).1'].isnull())].index
data['O2 Sat (%)'].loc[idx] = data['O2 Sat (%).1'].loc[idx].values

idx = data[(data['Respiratory Rate'].isnull()) & ~(data['Respiratory Rate.1'].isnull())].index
data['Respiratory Rate'].loc[idx] = data['Respiratory Rate.1'].loc[idx].values

data = data.drop(['Temperature.1', 'PEEP.1', 'FIO2 (%)', 'O2 Sat (%).1', 'Respiratory Rate.1'], axis = 1)
data = data.rename(columns={'FIO2 (%).1':'FIO2 (%)'})

In [200]:
cal = data[['patientunitstayid', 'weight']]
drop_stay_id = cal[cal['weight']==0].patientunitstayid.unique()
data_dropweightna = data[~(data.patientunitstayid.isin(drop_stay_id))]
data = data_dropweightna.drop('weight_fillna', axis = 1)

In [201]:
data['med_Norepinephrine (ml)'] = data['med_Norepinephrine (ml)'] + data['med_Norepinephrine (ml).1']
data['med_Dopamine (ml)'] = data['med_Dopamine (ml)'] + data['med_Dopamine (ml).1']

data = data.drop(['med_Norepinephrine (ml).1', 'med_Dopamine (ml).1', 'med_Dopamine (ml)_rate', 'med_Norepinephrine (ml)_rate'], axis = 1)

In [202]:
# unit conversion

def ml2mg(df, variable_ml): #ml 2 mg
    
    dataset = df.copy()
    format='2_mg'
    
    if( variable_ml == 'Norepinephrine (ml)') | (variable_ml == 'med_Norepinephrine (ml)'): # g/cm^3
        density = 1.4
        scale = 1000
    elif (variable_ml == 'Epinephrine (ml)') | (variable_ml == 'med_Epinephrine (ml)'): # g/cm^3
        density = 1.3
        scale = 1000
        
    elif (variable_ml == 'Dopamine (ml)') | (variable_ml == 'med_Dopamine (ml)'): # g/cm^3
        density = 1.26
        scale = 1000
        
    elif (variable_ml == 'Dobutamine (ml)') | (variable_ml == 'med_Dobutamine (ml)'): # g/ml
        density = 12.5
        scale = 1
        
    elif (variable_ml == 'Phenylephrine (ml)') | (variable_ml == 'med_Phenylephrine (ml)'): # g/cm3
        density = 1.275
        scale = 1000
    
    elif variable_ml == 'Vasopressin (ml)': # g/cm3
        density = 1.6
        scale = 1000
        
    elif (variable_ml == 'Labetalol (ml)') | (variable_ml == 'med_Labetalol(ml)'): # g/cm3
        density = 1.2
        scale = 1000
        
    elif (variable_ml =='med_Milrinone (ml)') | (variable_ml == 'Milrinone (ml)'): # g/cm3
        density = 1.344 
        scale = 1000
    
    dataset[variable_ml + f"_{format}"] = dataset[variable_ml] * density / scale
    
    return dataset[[variable_ml + f"_{format}"]]
    
def mcg2mg(df, variable_mcg): #mcg 2 mg
    
    dataset = df.copy()
    format='2_mg'
    
    dataset[variable_mcg + f"_{format}"] = dataset[variable_mcg] / 1000
    return dataset[[variable_mcg + f"_{format}"]]
    
def mcgkg2mg(df, variable_mcgkg): #mcg/kg 2 mg
    
    dataset = df.copy()
    format='2_mg'

    dataset[variable_mcgkg + f"_{format}"] = (dataset[variable_mcgkg] * 1000) * dataset['weight']
    
    return dataset[[variable_mcgkg + f"_{format}"]]

def unitkg2mg(df, variable_unitkg): #unit/kg 2 mg
    
    dataset = df.copy()
    format='2_mg'

    dataset[variable_unitkg + f"_{format}"] = ((dataset[variable_unitkg]/530)) * dataset['weight']
    
    return dataset[variable_unitkg + f"_{format}"]

def unit2mg(df, variable_unit): #unit 2 mg
    
    dataset = df.copy()
    format='2_mg'

    dataset[variable_unit + f"_{format}"] = dataset[variable_unit] / 530
    
    return dataset[variable_unit + f"_{format}"]

def ml2unit(df, variable_ml): #ml 2 unit
    
    dataset = df.copy()
    format = '2_unit'
    dataset[variable_ml + f"_{format}"] = dataset[variable_ml] / 100
    return dataset[variable_ml+f"_{format}"] 

def mcg2unit(df, variable_mcg): #mcg 2 unit
    
    dataset = df.copy()
    format = '2_unit'
    dataset[variable_mcg + f"_{format}"] = dataset[variable_mcg] * 0.0256
    return dataset[variable_mcg+f"_{format}"] 
    
    
def Rmlhr2mcgkgmin(df, variable_ml_rate): #rate ml/hr 2 mcg/kg/min
    dataset = df.copy()
    format='2_mcg/kg/min'
    
    if variable_ml_rate == 'Norepinephrine (ml)_rate': # g/cm^3
        density = 1.4
        scale = 1000
    
    elif variable_ml_rate == 'Epinephrine (ml)_rate': # g/cm^3
        density = 1.3
        scale = 1000

    elif variable_ml_rate == 'Dopamine (ml)_rate': # g/cm^3
        density = 1.26
        scale = 1000

    elif variable_ml_rate == 'Dobutamine (ml)_rate': # g/ml
        density = 12.5
        scale = 1
        
    elif variable_ml_rate == 'Phenylephrine (ml)_rate': # g/cm3
        density = 1.275
        scale = 1000
        
    elif variable_ml_rate == 'Vasopressin (ml)_rate': # g/cm3
        density = 1.6
        scale = 1000    
        
    elif variable_ml_rate == 'Milrinone (ml)_rate' : # g/cm3
        density = 1.344 
        scale = 1000 
        
    time = 60
    dataset[variable_ml_rate + f"_{format}"] = dataset[variable_ml_rate] * time * density * scale
    
    return dataset[variable_ml_rate + f"_{format}"]
    
def Rmcgmin2mcgkgmin(df, variable_mcg): #rate mcg/min 2 mcg/kg/min
    dataset = df.copy()
    format='2_mcg/kg/min'
    
    dataset[variable_mcg + f"_{format}"] = dataset[variable_mcg] / dataset['weight']
    
    return dataset[variable_mcg + f"_{format}"]
    
def Rmghr2mcgkgmin(df, variable_mg): #rate mg/hr 2 mcg/kg/min
    dataset = df.copy()
    format='2_mcg/kg/min'
    
    dataset[variable_mg + f"_{format}"] = ((dataset[variable_mg] * 1000) / dataset['weight']) * 60
    
    return dataset[variable_mg + f"_{format}"]

def Runit2mcgkgmin(df, variable_unit): #rate unit/hr 2 mcg/kg/min
    dataset = df.copy()
    format='2_mcg/kg/min'
    
    dataset[variable_unit + f"_{format}"] = ((dataset[variable_unit]/530) * 1000 / dataset['weight']) * 60
    
    return dataset[variable_unit + f"_{format}"]

In [203]:
def unit_conversion(search_original_element, data):

    df = data.copy()
    
    if search_original_element == 'epine':
        matching_columns = [col for col in df.columns if 'epine' in col.lower() and 'norepine' not in col.lower()]

    else:
        matching_columns = [col for col in df.columns if search_original_element.lower() in col.lower()]

    amount_mg = pd.DataFrame()
    rate_mcgkgmin = pd.DataFrame()

    for item in matching_columns:
        gc.collect()
        if '(' + item.split('(')[-1] == '(ml)':
            amount_mg = pd.concat([amount_mg, ml2mg(df, item)], axis = 1)
            
        elif '(' + item.split('(')[-1] == '(mcg)':
            amount_mg = pd.concat([amount_mg, mcg2mg(df, item)], axis = 1)
            
        elif '(' + item.split('(')[-1] == '(mcg/kg)':
            amount_mg = pd.concat([amount_mg, mcgkg2mg(df, item)], axis = 1)
            
        elif '(' + item.split('(')[-1] == '(ml)_rate':
            rate_mcgkgmin = pd.concat([rate_mcgkgmin, Rmlhr2mcgkgmin(df, item)], axis = 1)
        
        elif '(' + item.split('(')[-1] == '(mcg)_rate':
            rate_mcgkgmin = pd.concat([rate_mcgkgmin, Rmcgmin2mcgkgmin(df, item)], axis = 1)
            
        elif '(' + item.split('(')[-1] == '(mg)_rate':
            rate_mcgkgmin = pd.concat([rate_mcgkgmin, Rmghr2mcgkgmin(df, item)], axis = 1)    
        
        
    return matching_columns, amount_mg, rate_mcgkgmin

def unit_conversion_insulin(search_original_element, data):
    df = data.copy()
    
    matching_columns = [col for col in df.columns if search_original_element.lower() in col.lower()]
    
    amount_unit = pd.DataFrame()
    
    for item in matching_columns:
        if '(' + item.split('(')[-1] == '(ml)':
            amount_unit = pd.concat([amount_unit, ml2unit(df, item)], axis = 1)
            
        if '(' + item.split('(')[-1] == '(mcg)':
            amount_unit = pd.concat([amount_unit, mcg2unit(df, item)], axis = 1)
            
        return matching_columns, amount_unit

In [204]:
target = data.reset_index(drop=True)

drop_col, amount_nore, rate_nore = unit_conversion('nore', target)
target['Norepinephrine (MG)'] = amount_nore.sum(axis=1) + target['Norepinephrine (mg)']
target['Norepinephrine (MCG/KG/MIN)_Rate'] = rate_nore.sum(axis=1) + target['Norepinephrine (mcg/kg)_rate']

target = target.drop(drop_col, axis = 1)

drop_col, amount_epine, rate_epine = unit_conversion('epine', target)
target['Epinephrine (MG)'] = amount_epine.sum(axis=1) + target['Epinephrine (mg)']
target['Epinephrine (MCG/KG/MIN)_Rate'] = rate_epine.sum(axis=1) + target['Epinephrine (mcg/kg)_rate']

target = target.drop(drop_col, axis = 1)

drop_col, amount_dopa, rate_dopa = unit_conversion('dopamin', target)
target['Dopamine (MG)'] = amount_dopa.sum(axis=1)
target['Dopamine (MCG/KG/MIN)_Rate'] = rate_dopa.sum(axis=1) + target['Dopamine (mcg/kg)_rate']

target = target.drop(drop_col, axis = 1)

drop_col, amount_dobu, rate_dobu = unit_conversion('dobuta', target)
target['Dobutamine (MG)'] = amount_dobu.sum(axis=1)
target['Dobutamine (MCG/KG/MIN)_Rate'] = rate_dobu.sum(axis=1) + target['Dobutamine (mcg/kg)_rate']

target = target.drop(drop_col, axis = 1)

drop_col, amount_phen, rate_phen = unit_conversion('phenyl', target)
target['Phenylephrine (MG)'] = amount_phen.sum(axis=1)
target['Phenylephrine (MCG/KG/MIN)_Rate'] = rate_phen.sum(axis=1) + target['Phenylephrine (mcg/kg)_rate']

target = target.drop(drop_col, axis = 1)

drop_col, amount_vaso, rate_vaso = unit_conversion('vasopre', target)
target['Vasopressin (MG)'] = amount_vaso.sum(axis=1) + target['Vasopressin (mg)']
target['Vasopressin (MCG/KG/MIN)_Rate'] = rate_vaso.sum(axis=1) + target['Vasopressin (mcg/kg)_rate']

target = target.drop(drop_col, axis = 1)

target = target.drop(['Labetalol', 'Labetalol (ml)_rate', 'Labetalol(mg)_rate', 'Labetalol (mg)_rate', 'Labetalol_rate'], axis = 1)

drop_col, amount_labeta, _ = unit_conversion('labeta', target)
target['Labetalol (MG)'] = amount_labeta.sum(axis=1) + target['Labetalol(mg)'] + target['Labetalol (mg)'] 

target = target.drop(drop_col, axis = 1)

target = target.rename(columns = {'med_Esmolol':'Esmolol (MG)'})
target = target.rename(columns = {'med_Water':'Water'})

drop_col, amount_milri, _ = unit_conversion('Milrino', target)
target['Milrinone (MG)'] = amount_milri.sum(axis=1)
target = target.drop(drop_col, axis = 1)

drop_col, amount_insulin = unit_conversion_insulin('insulin', target)
target['Insulin (Unit)'] = amount_insulin.sum(axis=1)
target = target.drop(drop_col, axis = 1)

target = target.drop('Enteral Nutrition Ingredient_rate', axis = 1)
matching_columns = [col for col in target.columns if 'Nutrition'.lower() in col.lower()]
target['Nutrition (ml)'] = target[matching_columns].sum(axis=1)
target = target.drop(matching_columns, axis = 1)

matching_columns = [col for col in target.columns if 'Nacl'.lower() in col.lower()]
target['Nacl 3%'] = target[['med_NaCl 3%', 'NaCl 3%']].sum(axis=1)
target['Nacl 0.9%'] = target[['med_NaCl 0.9%', 'NaCl 0.9%']].sum(axis=1)
target['Nacl 0.45%'] = target[['med_NaCl 0.45%', 'NaCl 0.45%']].sum(axis=1)
target['Nacl 1.5%'] = target[['med_NaCl 1.5%']].sum(axis=1)
target = target.drop(matching_columns, axis = 1)

matching_columns = [col for col in target.columns if 'Nacl'.lower() in col.lower()]
target['Nacl n% (sum)'] = target[matching_columns].sum(axis = 1)
target = target.drop(matching_columns, axis = 1)

matching_columns = [col for col in target.columns if 'dextro'.lower() in col.lower()]
target['Dextrose n% (sum)'] = target[matching_columns].sum(axis=1)
target = target.drop(matching_columns, axis = 1)

drop_col, amount_hepa = unit_conversion_insulin('Heparin', target)
target['Heparin sodium (Unit)'] = amount_hepa.sum(axis=1)
target = target.drop(drop_col, axis = 1)

In [205]:
idx = target[target['Temperature']>50].index
target['Temperature'].loc[idx] = np.nan

In [206]:
target['Antibiotics'] = target['med_Antibiotics (ml)'].apply(lambda x: 1 if not pd.isna(x) and x > 0 else 0)
target = target.drop('med_Antibiotics (ml)', axis = 1)

In [207]:
null_dict = {'NIBPs' : 125, 'O2 Sat (%)' : 65, 'Respiratory Rate' : 12, 'HR' : 70,
            'NIBPd' : 75, 'SaO2' : 94, 'Temperature': 36.5, 'RASS' : 0, 'ABPs': 125, 'PEEP' : 0, 'Peak Insp. Pressure':0,
            'ABPd':75, 'CVP':4, 'Verbal Response': 5, 'Motor Response':6, 'EtCO2': 40, 'Eye Opening':4,
            'SVO2':70, 'Stroke Volume': 50, 'PAOP':20, 'SpO2': 95, 'ECMO':0, 'Lactate':1.1, 'Sodium':135,
            'Creatinine':0.7, 'Anion gap' : 10 , 'Glucose':5 , 'Potassium':4, 'pH':7, 'PaCO2':40, 'Ca+':8.5, 'Hemoglobin':12,
            'PaO2':75, 'Alkaline phosphatase':85, 'BUN':23, 'ALT':25, 'AST':25, 'FIO2 (%)':21, 'CO2':25, 'INR':1,
            'Total Bilirubin':2, 'RedBloodCell':3.5, 'Hematocrit':42, 'Platelets':300, 'WBC':10, 'Tropinin-T':0,
            'CRP':4}

def default_imputation(targ, null_dict):
    for col, default_value in null_dict.items():
        # Create a new column to track imputed values
        imputed_col_name = col + '_fillna'
        targ[imputed_col_name] = targ[col].isnull().astype(int)
        
        # Fill missing values
        targ[col] = targ[col].fillna(default_value)
        
def Imputation_process(df, null_dict):
    targ = df.copy()
    default_imputation(targ, null_dict)
    
    idx_map = targ[targ['MAP'].isnull()].index
    idx_co = targ[targ['Cardiac Output'].isnull()].index
    
    imputed_col_name = 'MAP' + '_fillna'
    targ[imputed_col_name] = targ['MAP'].isnull().astype(int)
    targ['MAP'].loc[idx_map] = (2*targ['ABPd'].loc[idx_map]+targ['ABPs'].loc[idx_map])/3
    
    imputed_col_name = 'Cardiac Output' + '_fillna'
    targ[imputed_col_name] = targ['Cardiac Output'].isnull().astype(int)
    targ['Cardiac Output'].loc[idx_co] = 3.5*0.007184 * (targ['weight'].loc[idx_co]**0.425) * (targ['height'].loc[idx_co]**0.725)
    
    targ['PaO2/FiO2'] = targ['PaO2']/(targ['FIO2 (%)']/100)
    
    return targ

In [208]:
imputed = Imputation_process(target, null_dict)

In [209]:
imputed

,Time_since_ICU_admission,med_Nitroprusside,Water,Esmolol (MG),IABP,CXR,CT scan,Blood culture,MRI,Catheter,Antibiotic(lab),Urine output,NIBPs,O2 Sat (%),Respiratory Rate,HR,MAP,NIBPd,SaO2,Temperature,RASS,ABPs,PEEP,Peak Insp. Pressure,ABPd,CVP,Verbal Response,Motor Response,EtCO2,Eye Opening,SVO2,Stroke Volume,Cardiac Output,PAOP,SpO2,ECMO,Lactate,Sodium,Creatinine,Anion gap,Glucose,Potassium,pH,PaCO2,Ca+,Hemoglobin,PaO2,Alkaline phosphatase,BUN,ALT,AST,FIO2 (%),CO2,INR,Total Bilirubin,RedBloodCell,Hematocrit,Platelets,WBC,Tropinin-T,CRP,Ventilator,height,weight,uniquepid,patientunitstayid,Age,gender,ethnicity,height_fillna,Norepinephrine (MG),Norepinephrine (MCG/KG/MIN)_Rate,Epinephrine (MG),Epinephrine (MCG/KG/MIN)_Rate,Dopamine (MG),Dopamine (MCG/KG/MIN)_Rate,Dobutamine (MG),Dobutamine (MCG/KG/MIN)_Rate,Phenylephrine (MG),Phenylephrine (MCG/KG/MIN)_Rate,Vasopressin (MG),Vasopressin (MCG/KG/MIN)_Rate,Labetalol (MG),Milrinone (MG),Insulin (Unit),Nutrition (ml),Nacl n% (sum),Dextrose n% (sum),Heparin sodium (Unit),Antibiotics,NIBPs_fillna,O2 Sat (%)_fillna,Respiratory Rate_fillna,HR_fillna,NIBPd_fillna,SaO2_fillna,Temperature_fillna,RASS_fillna,ABPs_fillna,PEEP_fillna,Peak Insp. Pressure_fillna,ABPd_fillna,CVP_fillna,Verbal Response_fillna,Motor Response_fillna,EtCO2_fillna,Eye Opening_fillna,SVO2_fillna,Stroke Volume_fillna,PAOP_fillna,SpO2_fillna,ECMO_fillna,Lactate_fillna,Sodium_fillna,Creatinine_fillna,Anion gap_fillna,Glucose_fillna,Potassium_fillna,pH_fillna,PaCO2_fillna,Ca+_fillna,Hemoglobin_fillna,PaO2_fillna,Alkaline phosphatase_fillna,BUN_fillna,ALT_fillna,AST_fillna,FIO2 (%)_fillna,CO2_fillna,INR_fillna,Total Bilirubin_fillna,RedBloodCell_fillna,Hematocrit_fillna,Platelets_fillna,WBC_fillna,Tropinin-T_fillna,CRP_fillna,MAP_fillna,Cardiac Output_fillna,PaO2/FiO2
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.000000,94.000000,19.833333,90.500000,91.666667,49.000000,95.200000,36.5,0.0,125.000000,0.0,0.0,75.0,4.0,5.0,6.0,40.0,4.0,70.0,50.0,6.114209,20.0,95.0,0.0,1.1,135.0,0.70,10.0,5.0,4.0,7.0,40.0,8.5,12.0,75.0,85.0,23.0,25.0,25.0,21.0,25.0,1.00,2.0,3.5,42.0,300.0,10.0,0.0,4.00,0.0,162.6,69.5,002-10300,165269,53,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,357.142857
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.250000,94.666667,19.933333,90.400000,91.666667,48.000000,95.000000,36.5,0.0,125.000000,0.0,0.0,75.0,4.0,4.0,5.0,40.0,3.0,70.0,50.0,6.114209,20.0,95.0,0.0,1.1,135.0,0.70,10.0,5.0,4.0,7.0,40.0,8.5,12.0,75.0,85.0,23.0,25.0,25.0,21.0,25.0,1.00,2.0,3.5,42.0,300.0,10.0,0.0,4.00,0.0,162.6,69.5,002-10300,165269,53,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,357.142857
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.400000,96.250000,19.937500,90.062500,91.666667,54.400000,95.750000,36.5,0.0,125.000000,0.0,0.0,75.0,19.0,4.0,5.0,40.0,3.0,70.0,50.0,6.114209,20.0,95.0,0.0,1.1,135.0,0.70,10.0,5.0,4.0,7.0,40.0,8.5,12.0,75.0,85.0,23.0,25.0,25.0,21.0,25.0,1.00,2.0,3.5,42.0,300.0,10.0,0.0,4.00,0.0,162.6,69.5,002-10300,165269,53,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,357.142857
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.500000,97.500000,22.846154,91.583333,91.666667,58.000000,97.125000,36.5,0.0,125.000000,0.0,0.0,75.0,19.0,4.0,5.0,40.0,3.0,70.0,50.0,6.114209,20.0,95.0,0.0,1.1,135.0,0.70,10.0,5.0,4.0,7.0,40.0,8.5,12.0,75.0,85.0,23.0,25.0,25.0,21.0,25.0,1.00,2.0,3.5,42.0,300.0,10.0,0.0,4.00,0.0,162.6,69.5,002-10300,165269,53,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1

In [210]:
vaso_ino = ['Norepinephrine (MG)', 'Epinephrine (MG)', 'Dopamine (MG)', 'Dobutamine (MG)', 'Phenylephrine (MG)', 'Vasopressin (MG)', 'Milrinone (MG)','med_Nitroprusside']

imputed['vasoactive/inotropic'] = imputed[vaso_ino].sum(axis = 1)
imputed['vasoactive/inotropic'] = imputed['vasoactive/inotropic'].apply(lambda x: 1 if not pd.isna(x) and x > 0 else 0)

In [211]:
print('총 환자 수 ', len(imputed.uniquepid.unique()))
print('총 입원 수 ', len(imputed.patientunitstayid.unique())) 
print('관측치 수 ', len(imputed))

총 환자 수  13405
총 입원 수  14236
관측치 수  2506203


In [212]:
imputed['ACE_inhibitors(beta-blocker)'] = imputed['Labetalol (MG)'] + imputed['Esmolol (MG)']
imputed['ACE_inhibitors(beta-blocker)'] = imputed['ACE_inhibitors(beta-blocker)'].apply(lambda x: 1 if not pd.isna(x) and x > 0 else 0)

In [213]:
imputed['ACE_inhibitors(beta-blocker)'].value_counts()

0    2505026
1       1177
Name: ACE_inhibitors(beta-blocker), dtype: int64

In [214]:
imputed = imputed.drop(['Labetalol (MG)','Esmolol (MG)', 'ACE_inhibitors(beta-blocker)'], axis = 1)

In [215]:
imputed['Heparin sodium (Unit)'] = imputed['Heparin sodium (Unit)']* 100 
imputed['Fluids(ml)'] = imputed[['Insulin (Unit)', 'Nutrition (ml)', 'Nacl n% (sum)', 'Dextrose n% (sum)', 'Heparin sodium (Unit)', 'Water']].sum(axis=1)
imputed = imputed.drop(['Insulin (Unit)', 'Nutrition (ml)', 'Nacl n% (sum)', 'Dextrose n% (sum)', 'Heparin sodium (Unit)', 'Water'], axis = 1)

In [75]:
imputed

,Time_since_ICU_admission,med_Nitroprusside,IABP,CXR,CT scan,Blood culture,MRI,Catheter,Antibiotic(lab),Urine output,NIBPs,O2 Sat (%),Respiratory Rate,HR,MAP,NIBPd,SaO2,Temperature,RASS,ABPs,PEEP,Peak Insp. Pressure,ABPd,CVP,Verbal Response,Motor Response,EtCO2,Eye Opening,SVO2,Stroke Volume,Cardiac Output,PAOP,SpO2,ECMO,Lactate,Sodium,Creatinine,Anion gap,Glucose,Potassium,pH,PaCO2,Ca+,Hemoglobin,PaO2,Alkaline phosphatase,BUN,ALT,AST,FIO2 (%),CO2,INR,Total Bilirubin,RedBloodCell,Hematocrit,Platelets,WBC,Tropinin-T,CRP,Ventilator,height,weight,uniquepid,patientunitstayid,Age,gender,ethnicity,height_fillna,Norepinephrine (MG),Norepinephrine (MCG/KG/MIN)_Rate,Epinephrine (MG),Epinephrine (MCG/KG/MIN)_Rate,Dopamine (MG),Dopamine (MCG/KG/MIN)_Rate,Dobutamine (MG),Dobutamine (MCG/KG/MIN)_Rate,Phenylephrine (MG),Phenylephrine (MCG/KG/MIN)_Rate,Vasopressin (MG),Vasopressin (MCG/KG/MIN)_Rate,Milrinone (MG),Antibiotics,NIBPs_fillna,O2 Sat (%)_fillna,Respiratory Rate_fillna,HR_fillna,NIBPd_fillna,SaO2_fillna,Temperature_fillna,RASS_fillna,ABPs_fillna,PEEP_fillna,Peak Insp. Pressure_fillna,ABPd_fillna,CVP_fillna,Verbal Response_fillna,Motor Response_fillna,EtCO2_fillna,Eye Opening_fillna,SVO2_fillna,Stroke Volume_fillna,PAOP_fillna,SpO2_fillna,ECMO_fillna,Lactate_fillna,Sodium_fillna,Creatinine_fillna,Anion gap_fillna,Glucose_fillna,Potassium_fillna,pH_fillna,PaCO2_fillna,Ca+_fillna,Hemoglobin_fillna,PaO2_fillna,Alkaline phosphatase_fillna,BUN_fillna,ALT_fillna,AST_fillna,FIO2 (%)_fillna,CO2_fillna,INR_fillna,Total Bilirubin_fillna,RedBloodCell_fillna,Hematocrit_fillna,Platelets_fillna,WBC_fillna,Tropinin-T_fillna,CRP_fillna,MAP_fillna,Cardiac Output_fillna,PaO2/FiO2,vasoactive/inotropic,Fluids(ml)
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.000000,94.000000,19.833333,90.500000,91.666667,49.000000,95.200000,36.5,0.0,125.000000,0.0,0.0,75.0,4.0,5.0,6.0,40.0,4.0,70.0,50.0,6.114209,20.0,95.0,0.0,1.1,135.0,0.70,10.0,5.0,4.0,7.0,40.0,8.5,12.0,75.0,85.0,23.0,25.0,25.0,21.0,25.0,1.00,2.0,3.5,42.0,300.0,10.0,0.0,4.00,0.0,162.6,69.5,002-10300,165269,53,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,357.142857,0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.250000,94.666667,19.933333,90.400000,91.666667,48.000000,95.000000,36.5,0.0,125.000000,0.0,0.0,75.0,4.0,4.0,5.0,40.0,3.0,70.0,50.0,6.114209,20.0,95.0,0.0,1.1,135.0,0.70,10.0,5.0,4.0,7.0,40.0,8.5,12.0,75.0,85.0,23.0,25.0,25.0,21.0,25.0,1.00,2.0,3.5,42.0,300.0,10.0,0.0,4.00,0.0,162.6,69.5,002-10300,165269,53,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,357.142857,0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99.400000,96.250000,19.937500,90.062500,91.666667,54.400000,95.750000,36.5,0.0,125.000000,0.0,0.0,75.0,19.0,4.0,5.0,40.0,3.0,70.0,50.0,6.114209,20.0,95.0,0.0,1.1,135.0,0.70,10.0,5.0,4.0,7.0,40.0,8.5,12.0,75.0,85.0,23.0,25.0,25.0,21.0,25.0,1.00,2.0,3.5,42.0,300.0,10.0,0.0,4.00,0.0,162.6,69.5,002-10300,165269,53,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,357.142857,0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.500000,97.500000,22.846154,91.583333,91.666667,58.000000,97.125000,36.5,0.0,125.000000,0.0,0.0,75.0,19.0,4.0,5.0,40.0,3.0,70.0,50.0,6.114209,20.0,95.0,0.0,1.1,135.0,0.70,10.0,5.0,4.0,7.0,40.0,8.5,12.0,75.0,85.0,23.0,25.0,25.0,21.0,25.0,1.00,2.0,3.5,42.0,300.0,10.0,0.0,4.00,0.0,162.6,69.5,002-10300,165269,53,1,2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,357.142857,0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,111.000000,97.500000,22.000000,91.000000,91.666667,56.000000,97.125000,36.5,0.0,125.000000,0.0,0.0,75.0,19.0,4.0,5.0,40.0,3.0,70.0,50.0,

In [216]:
intakeoutput = pd.read_csv("/Users/DAHS/Desktop/early_prediction_of_circ_scl/eicu-crd/2.0/intakeOutput.csv.gz", compression='gzip')

In [217]:
sample = intakeoutput[intakeoutput['cellpath'].str.contains('intake', case=False, regex=True)]

In [218]:
data = pd.read_csv('check_point_data.csv.gz', compression = 'gzip')

In [219]:
# Crystalloids

sample = intakeoutput[intakeoutput['cellpath'].str.contains('intake', case=False, regex=True)]
Crystalloids = sample[sample['celllabel']== 'Crystalloids']

Crystalloids=Crystalloids[Crystalloids['patientunitstayid'].isin(imputed['patientunitstayid'])]
Crystalloids=Crystalloids[Crystalloids['intakeoutputoffset'] >= 0]

Crystalloids = Crystalloids[['patientunitstayid', 'intakeoutputoffset', 'celllabel', 'cellvaluenumeric']]

Crystalloids = Crystalloids.rename(columns = {'intakeoutputoffset': 'start_time'})

Crystalloids['start_time']=np.rint(Crystalloids['start_time']/60)

Crystalloids['stop_time'] = Crystalloids['start_time'] + 1

#####Sanity check
Crystalloids['sanity']=Crystalloids['stop_time']-Crystalloids['start_time']
Crystalloids=Crystalloids[Crystalloids['sanity']>0]
del Crystalloids['sanity']
#####Select hadm_id as in main file
Crystalloids=Crystalloids[Crystalloids['patientunitstayid'].isin(data['patientunitstayid'])]
Crystalloids=pd.merge(Crystalloids,data[['patientunitstayid','los(hour)']],on='patientunitstayid',how='left')

#####Remove where start time is after end of visit
Crystalloids['sanity']=Crystalloids['los(hour)']-Crystalloids['start_time']
Crystalloids=Crystalloids[Crystalloids['sanity']>0]
del Crystalloids['sanity']
####Any stop_time after end of visit is set at end of visit
Crystalloids.loc[Crystalloids['stop_time'] > Crystalloids['los(hour)'],'stop_time']=Crystalloids.loc[Crystalloids['stop_time'] > Crystalloids['los(hour)'],'los(hour)']
del Crystalloids['los(hour)']

Crystalloids['cellvaluenumeric']=Crystalloids['cellvaluenumeric'].apply(pd.to_numeric, errors='coerce')

In [220]:
add_fluids = pd.DataFrame()

for hid in tqdm(imputed.patientunitstayid.unique(), desc = 'Tabularize EHR for total stay 14,236'):
    gc.collect()
    grp=data[data['patientunitstayid']==hid]
    los = int(grp['los(hour)'].values)
    height = grp['admissionheight'].values[0]
    weight = grp['admissionweight'].values[0]

    dyn_csv=pd.DataFrame()
    
    feat=Crystalloids['celllabel'].unique()
    df2=Crystalloids[Crystalloids['patientunitstayid']==hid]
    if df2.shape[0]==0:
        dose=pd.DataFrame(np.zeros([los,len(feat)]),columns=feat)
        dose=dose.fillna(0)
        dose.columns=pd.MultiIndex.from_product([["MEDS"], dose.columns])
    else:
        dose=df2.pivot_table(index='start_time',columns='celllabel',values='cellvaluenumeric')
        df2=df2.pivot_table(index='start_time',columns='celllabel',values='stop_time') #value는 큰 의미 없음

        add_indices = pd.Index(range(los)).difference(df2.index) # 처방 된 시간과 los 비교 후 처방이 이루어지지 않은 시간 포인트만큼 시간 인덱스 생성
        add_df = pd.DataFrame(index=add_indices, columns=df2.columns).fillna(np.nan) 
        df2=pd.concat([df2, add_df])
        df2=df2.sort_index()
        df2=df2.ffill()
        df2=df2.fillna(0)

        dose=pd.concat([dose, add_df])
        dose=dose.sort_index()
        dose=dose.ffill()
        dose=dose.fillna(0)

        df2.iloc[:,0:]=df2.iloc[:,0:].sub(df2.index,0) #end time - start time
        df2[df2>0]=1 # 약물 처방 시간 만큼 1
        df2[df2<0]=0 #약을 처방 받지 않은 경우에는 0으로 채웠었기 때문에 sub 연산시 음 값을 가지게 되어 0으로 변환됨

        dose.iloc[:,0:]=df2.iloc[:,0:]*dose.iloc[:,0:] # 실제 처방 된 경우의 값만 살아 남음
        feat_df=pd.DataFrame(columns=list(set(feat)-set(dose.columns)))
        dose=pd.concat([dose,feat_df],axis=1)


        dose=dose[feat]
        dose=dose.fillna(0)
        dose.columns=pd.MultiIndex.from_product([["MEDS"], dose.columns])

        
    if(dyn_csv.empty):
        dyn_csv=dose
        
    dyn_csv['patientunitstayid'] = hid
    add_fluids=pd.concat([add_fluids,dyn_csv],axis=0)

Tabularize EHR for total stay 14,236:   0%|          | 0/14236 [00:00<?, ?it/s]

Tabularize EHR for total stay 14,236: 100%|██████████| 14236/14236 [23:40<00:00, 10.02it/s]


In [221]:
add_fluids.columns=add_fluids.columns.droplevel(0)

In [222]:
imputed['Fluids(ml)'] = imputed['Fluids(ml)'] + add_fluids.reset_index(drop=True)['Crystalloids']

In [223]:
diagnosis = pd.read_csv("/Users/DAHS/Desktop/early_prediction_of_circ_scl/eicu-crd/2.0/diagnosis.csv.gz", compression = 'gzip')

In [224]:
diag = diagnosis[['patientunitstayid', 'diagnosisoffset', 'icd9code']]

In [225]:
# 여기서 부터 다양한 ICD 코드 발급 여부를 추가하면 됨!!

diag

,patientunitstayid,diagnosisoffset,icd9code
0,141168,72,"414.00, I25.10"
1,141168,118,NaN
2,141168,72,"491.20, J44.9"
3,141168,118,"491.20, J44.9"
4,141168,118,"428.0, I50.9"
...,...,...,...
2710667,3353251,11304,"599.0, N39.0"
2710668,3353251,4080,"038.9, A41.9"
2710669,3353254,41,"584.9, N17.9"
2710670,3353254,41,"578.9, K92.2"


In [226]:
def process_icd9code(code):
    if pd.isna(code):
        return np.nan
    parts = code.split(',')
    return parts[1].strip() if len(parts) > 1 else np.nan

# Apply the function to the 'icd9code' column
diag['icd9code_10'] = diag['icd9code'].apply(process_icd9code)

# Drop rows with NaN values
diag.dropna(subset=['icd9code_10'], inplace=True)

# Reset index if needed
diag.reset_index(drop=True, inplace=True)

In [227]:
def process_icd9code_first(code):
    if pd.isna(code):
        return np.nan
    parts = code.split(',')
    return parts[0].strip() if len(parts) > 0 else np.nan

# Apply the function to the 'icd9code' column
diag['icd9code_9'] = diag['icd9code'].apply(process_icd9code_first)

# Drop rows with NaN values
diag.dropna(subset=['icd9code'], inplace=True)

# Reset index if needed
diag.reset_index(drop=True, inplace=True)

In [228]:
diag['icd9code_9'] = diag['icd9code_9'].str.replace('.', '', regex=False)
diag['icd9code_10'] = diag['icd9code_10'].str.replace('.', '', regex=False)

In [229]:
Path = "/Users/DAHS/Desktop/early_prediction_of_circ_scl"
suspected_infection = pd.read_csv(Path+'/suspected_infection.csv')
si_icd10_ary = suspected_infection[suspected_infection['icd_version']==10]['icd_code'].values
si_icd10 = np.array(si_icd10_ary[0].split(', '))

In [230]:
si_icd9_ary = suspected_infection[suspected_infection['icd_version']==9]['icd_code'].values
si_icd9 = np.array(si_icd9_ary[0].split(', '))

In [231]:
suspected_infection_df = pd.DataFrame()

for icd in si_icd10:

    sample = diag[diag['icd9code_10'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    suspected_infection_df = pd.concat([suspected_infection_df, sample])
    
for icd in si_icd9:
    
    sample = diag[diag['icd9code_9'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    suspected_infection_df = pd.concat([suspected_infection_df, sample])
    
AMI_df = pd.DataFrame()

for icd in ['I21', 'I22']: #I21은 급성 심근경색을 나타내며, I22는 이전에 발생한 심근경색 후에 발생하는 재발성 심근경색을 의미

    sample = diag[diag['icd9code_10'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    AMI_df = pd.concat([AMI_df, sample])
    
# End-Stage Cardiomyopathy(심근증 말기)
    
ESC_df = pd.DataFrame()

for icd in ['I42']: #I42 코드는 심근증을 나타내며, 이 중 일부 하위 코드가 말기 심근증을 특정할 수 있음

    sample = diag[diag['icd9code_10'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    ESC_df = pd.concat([ESC_df, sample])
    
# Advanced Valvular Heart Disease(진행된 판막성 심장병)
    
AVHD_df = pd.DataFrame()

for icd in ['I34', 'I35', 'I36', 'I37', 'I38', 'I39']: #구체적인 진행 정도는 상세한 하위 코드를 통해 분류될 수 있음

    sample = diag[diag['icd9code_10'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    AVHD_df = pd.concat([AVHD_df, sample])
    
M_df = pd.DataFrame()

for icd in ['I40', 'I51.4']: #I40은 심근염을 직접 지칭, I51.4는 심장병의 합병증으로서의 심근염을 의미

    sample = diag[diag['icd9code_10'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    M_df = pd.concat([M_df, sample])
    
# Arrhythmia(부정맥)
    
A_df = pd.DataFrame()

for icd in ['I47', 'I48', 'I49']: #구체적인 진단에 따라 I47-I49 코드 중 적합한 코드가 사용됨

    sample = diag[diag['icd9code_10'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    A_df = pd.concat([A_df, sample])
    
# spinal cord injury(척수손상)
    
spinal_injury_before_df = pd.DataFrame()

for icd in ['G50', '805', '806']:

    sample = diag[diag['icd9code_10'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    spinal_injury_before_df = pd.concat([spinal_injury_before_df, sample])
    
# Pulmonary Embolism(폐색전증)
    
PE_df = pd.DataFrame()

for icd in ['I26']:

    sample = diag[diag['icd9code_10'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    PE_df = pd.concat([PE_df, sample])
    
# Tension Pneumothorax(긴장성 기흉)
    
TP_df = pd.DataFrame()

for icd in ['J93']:

    sample = diag[diag['icd9code_10'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    TP_df = pd.concat([TP_df, sample])
    
# 1. 외상에 의한 출혈(손상, 중독 및 특정 기타 외인에 의한 결과)
EX_df = pd.DataFrame()

for icd in ['S', 'T']:

    sample = diag[diag['icd9code_10'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    EX_df = pd.concat([EX_df, sample])

    
# 1. 내부에 의한 출혈(내상)(파열된 동맥류 등)
IN_df = pd.DataFrame()

for icd in [ 'I60','I61','I62','I63','I64','I65','I66','I67','I68','I69' ]:

    sample = diag[diag['icd9code_10'].str.startswith(icd)][['patientunitstayid','diagnosisoffset']]
    IN_df = pd.concat([IN_df, sample])

In [235]:
def make_categorical(df, variable_name, concat_df):
    data = concat_df.copy()
    data[variable_name] = 1
    data = data[['patientunitstayid', variable_name]].drop_duplicates()
    
    dicta = dict(zip(data.patientunitstayid, data.iloc[:, -1]))

    df[variable_name] = df['patientunitstayid'].copy()
    df[variable_name] = df[variable_name].map(dicta)
    df = df.fillna(0)
    return df
    
result1 = make_categorical(imputed, 'suspected_infection', suspected_infection_df)
result2 = make_categorical(result1, 'Acute_Myocardial_Infarction', AMI_df)
result3 = make_categorical(result2, 'End-Stage_Cardiomyopathy', ESC_df)
result4 = make_categorical(result3, 'Adv__Valvular_HD', AVHD_df)
result5 = make_categorical(result4, 'Myocarditis', A_df)
result6 = make_categorical(result5, 'spinal_cord_injury', spinal_injury_before_df)
result7 = make_categorical(result6, 'Pulmonary_Embolism', PE_df)
result8 = make_categorical(result7, 'Tension_Pneumothorax', TP_df)
result9 = make_categorical(result8, 'hypovolemia_external', EX_df)
result10 = make_categorical(result9, 'hypovolemia_internal', IN_df)

In [236]:
print(result10['suspected_infection'].value_counts())
print(result10['Acute_Myocardial_Infarction'].value_counts())
print(result10['End-Stage_Cardiomyopathy'].value_counts())
print(result10['Adv__Valvular_HD'].value_counts())
print(result10['Myocarditis'].value_counts())
print(result10['spinal_cord_injury'].value_counts())
print(result10['Pulmonary_Embolism'].value_counts())
print(result10['Tension_Pneumothorax'].value_counts())
print(result10['hypovolemia_external'].value_counts())
print(result10['hypovolemia_internal'].value_counts())

1.0    1334822
0.0    1171381
Name: suspected_infection, dtype: int64
0.0    2342290
1.0     163913
Name: Acute_Myocardial_Infarction, dtype: int64
0.0    2504796
1.0       1407
Name: End-Stage_Cardiomyopathy, dtype: int64
0.0    2451487
1.0      54716
Name: Adv__Valvular_HD, dtype: int64
0.0    2068098
1.0     438105
Name: Myocarditis, dtype: int64
0.0    2506203
Name: spinal_cord_injury, dtype: int64
0.0    2452700
1.0      53503
Name: Pulmonary_Embolism, dtype: int64
0.0    2499104
1.0       7099
Name: Tension_Pneumothorax, dtype: int64
0.0    2284964
1.0     221239
Name: hypovolemia_external, dtype: int64
0.0    2342987
1.0     163216
Name: hypovolemia_internal, dtype: int64


In [237]:
proc = pd.read_csv("/Users/DAHS/Desktop/early_prediction_of_circ_scl/eicu-crd/2.0/treatment.csv.gz", compression='gzip')

blist =['infectious diseases|cultures / immuno-assays|cultures|blood',
'infectious diseases|cultures / immuno-assays|cultures|blood|drawn from central line',
'infectious diseases|cultures / immuno-assays|cultures',
'infectious diseases|cultures / immuno-assays|cultures|blood|peripheral',
'infectious diseases|cultures / immuno-assays|cultures|catheter tip',
'infectious diseases|cultures / immuno-assays|cultures|BAL/PBS|bacterial',
'infectious diseases|cultures / immuno-assays|cultures|BAL/PBS|comprehensive (bacterial, viral, fungal, afb, etc.',
'surgery|infection|cultures|surgical specimen',
'infectious diseases|procedures|vascular catheter - change|tip cultured',
'surgery|infection|cultures|blood',
'infectious diseases|procedures|vascular catheter - removal|tip cultured',
'infectious diseases|cultures / immuno-assays|cultures|pericardial fluid',
'surgery|infection|cultures|blood|peripheral',
'infectious diseases|cultures / immuno-assays|warm-reactive antibody',
'infectious diseases|cultures / immuno-assays|cold-reactive antibody',
'infectious diseases|cultures / immuno-assays|cultures|catheter tip|quantitative']

add_bloodculture = proc[proc['treatmentstring'].isin(blist)]

add_bloodculture['blood_culture'] = 1
add_bloodculture = add_bloodculture[['patientunitstayid', 'blood_culture']].drop_duplicates()
blood_dict = dict(zip(add_bloodculture.patientunitstayid, add_bloodculture.blood_culture))

result10['blood_culture'] = result10['patientunitstayid'].copy()
result10['blood_culture'] = result10['blood_culture'].map(suspected_dict)
result10 = result10.fillna(0)

result10['Blood culture'] = result10[['blood_culture', 'Blood culture']].sum(axis=1)
result10 = result10.drop('blood_culture', axis = 1)
result10['Blood culture'] = result10['Blood culture'].apply(lambda x: 1 if not pd.isna(x) and x > 0 else 0)

In [238]:
result10['GCS_score'] = result10['Motor Response'] + result10['Eye Opening'] + result10['Verbal Response']

In [239]:
def make_sofa_score(df):
    sample = df.copy()
    
    sample['Sofa_Respiration'] = None       # Respiratory system
    sample['Sofa_Coagulation'] = None       # Coagulation system
    sample['Sofa_Liver'] = None             # Hepatic system
    sample['Sofa_Cardiovascular'] = None    # Cardiovascular system
    sample['Sofa_GCS'] = None               # Neurologic system
    sample['Sofa_Urine'] = None             # Renal system
    
    
    # Respiratory system
    score_0 = sample[sample['PaO2/FiO2'] >= 400].index
    score_1 = sample[(sample['PaO2/FiO2'] < 400) & (sample['PaO2/FiO2'] > 300)].index
    score_2 = sample[(sample['PaO2/FiO2'] <= 300) & (sample['PaO2/FiO2'] > 200)].index
    score_3 = sample[(sample['PaO2/FiO2'] <= 200) & ((sample['Ventilator'] > 0)) * (sample['PaO2/FiO2'] > 100)].index
    # Note: For score 4, we don't need to check for'PaO2/FiO2'< 200 since it's already included in score_3 condition.
    score_4 = sample[(sample['PaO2/FiO2'] <= 100) & ((sample['Ventilator'] > 0))].index
    
    sample.loc[score_0, 'Sofa_Respiration'] = 0
    sample.loc[score_1, 'Sofa_Respiration'] = 1
    sample.loc[score_2, 'Sofa_Respiration'] = 2
    sample.loc[score_3, 'Sofa_Respiration'] = 3
    sample.loc[score_4, 'Sofa_Respiration'] = 4
    sample = sample.fillna(0)

    # Coagulation system
    score_0 = sample[sample['Platelets'] > 150].index
    score_1 = sample[(sample['Platelets'] <= 150) & (sample['Platelets'] > 100)].index
    score_2 = sample[(sample['Platelets'] <= 100) & (sample['Platelets'] > 50)].index
    score_3 = sample[(sample['Platelets'] <= 50) & (sample['Platelets'] > 20)].index
    score_4 = sample[sample['Platelets'] <= 20].index
    
    sample.loc[score_0, 'Sofa_Coagulation'] = 0
    sample.loc[score_1, 'Sofa_Coagulation'] = 1
    sample.loc[score_2, 'Sofa_Coagulation'] = 2
    sample.loc[score_3, 'Sofa_Coagulation'] = 3
    sample.loc[score_4, 'Sofa_Coagulation'] = 4

    # Hepatic system
    score_0 = sample[sample['Total Bilirubin'] < 1.2].index
    score_1 = sample[(sample['Total Bilirubin'] >= 1.2) & (sample['Total Bilirubin'] <= 1.9)].index
    score_2 = sample[(sample['Total Bilirubin'] >= 2) & (sample['Total Bilirubin'] <= 5.9)].index
    score_3 = sample[(sample['Total Bilirubin'] >= 6) & (sample['Total Bilirubin'] <= 11.9)].index
    score_4 = sample[sample['Total Bilirubin'] >= 12].index
    
    sample.loc[score_0, 'Sofa_Liver'] = 0
    sample.loc[score_1, 'Sofa_Liver'] = 1
    sample.loc[score_2, 'Sofa_Liver'] = 2
    sample.loc[score_3, 'Sofa_Liver'] = 3
    sample.loc[score_4, 'Sofa_Liver'] = 4

    # Cardiovascular system
    score_0 = sample[sample['MAP'] >= 70].index
    score_1 = sample[(sample['MAP'] < 70)].index
    score_2 = sample[((sample['Dopamine (MCG/KG/MIN)_Rate'] <= 5)) | (sample['Dopamine (MG)'] > 0)].index
    score_3 = sample[((sample['Dopamine (MCG/KG/MIN)_Rate'] > 5) & (sample['Epinephrine (MCG/KG/MIN)_Rate'] <= 0.1))].index
    score_4 = sample[(sample['Dopamine (MCG/KG/MIN)_Rate'] > 15) | (sample['Epinephrine (MCG/KG/MIN)_Rate'] > 0.1) | (sample['Norepinephrine (MCG/KG/MIN)_Rate'] > 0.1)].index
    
    sample.loc[score_0, 'Sofa_Cardiovascular'] = 0
    sample.loc[score_1, 'Sofa_Cardiovascular'] = 1
    sample.loc[score_2, 'Sofa_Cardiovascular'] = 2
    sample.loc[score_3, 'Sofa_Cardiovascular'] = 3
    sample.loc[score_4, 'Sofa_Cardiovascular'] = 4

    # Neurologic system
    score_0 = sample[sample['GCS_score'] == 15].index
    score_1 = sample[(sample['GCS_score'] >= 13) & (sample['GCS_score'] <= 14)].index
    score_2 = sample[(sample['GCS_score'] >= 10) & (sample['GCS_score'] <= 12)].index
    score_3 = sample[(sample['GCS_score'] >= 6)  & (sample['GCS_score'] <= 9)].index
    score_4 = sample[sample['GCS_score']<6].index
    
    sample.loc[score_0, 'Sofa_GCS'] = 0
    sample.loc[score_1, 'Sofa_GCS'] = 1
    sample.loc[score_2, 'Sofa_GCS'] = 2
    sample.loc[score_3, 'Sofa_GCS'] = 3
    sample.loc[score_4, 'Sofa_GCS'] = 4 

    # Renal system
    
    # Creatinine < 1.2, score 0
    score_0 = sample[sample['Creatinine'] < 1.2].index
    print(len(score_0))

    # Creatinine 1.2 - 1.9, score 1
    score_1 = sample[(sample['Creatinine'] >= 1.2) & (sample['Creatinine'] <= 1.9)].index

    # Creatinine 2.0 - 3.4, score 2
    score_2 = sample[(sample['Creatinine'] >= 2.0) & (sample['Creatinine'] <= 3.4)].index

    # Creatinine 3.5 - 4.9 OR Urine 200 - 500, score 3
    score_3 = sample[((sample['Creatinine'] >= 3.5) & (sample['Creatinine'] <= 4.9)) |
                    ((sample['Urine output'] >= 200) & (sample['Urine output'] <= 500))].index

    # Creatinine > 5 OR Urine < 200, score 4
    score_4 = sample[(sample['Creatinine'] > 5) | (sample['Urine output'] < 200)].index


    # Assign scores to the 'Sofa_Urine' column based on the calculated indices
    sample.loc[score_4, 'Sofa_Urine'] = 4
    sample.loc[score_3, 'Sofa_Urine'] = 3
    sample.loc[score_2, 'Sofa_Urine'] = 2
    sample.loc[score_1, 'Sofa_Urine'] = 1
    sample.loc[score_0, 'Sofa_Urine'] = 0

    
    
    sample['SoFa_score'] = sample['Sofa_Respiration'] + sample['Sofa_Coagulation'] + sample['Sofa_Liver'] + sample['Sofa_Cardiovascular'] + sample['Sofa_GCS'] + sample['Sofa_Urine']
    
    # sample = sample.drop('pao2/fio2', axis = 1)
    
    return sample

after_sofa = make_sofa_score(result10)

1361239


In [240]:
def annotation(df):
    # DataFrame 복사본 생성
    targ = df.copy()

    targ['Annotation'] = np.nan

    # 각 stay_id에 대해 반복
    for stay_id in tqdm(targ['patientunitstayid'].unique()):
        stay_df = targ[targ['patientunitstayid'] == stay_id].sort_values(by='Time_since_ICU_admission')
        for idx, row in stay_df.iterrows():
            current_time = row['Time_since_ICU_admission']
            pastpoint_window_1h = current_time - 1
        
            # 1시간 이전 데이터 필터링
            relevant_rows = stay_df[(stay_df['Time_since_ICU_admission'] <= current_time) & (stay_df['Time_since_ICU_admission'] >= pastpoint_window_1h)]
            # 조건 확인
            no_circ_cond = (relevant_rows['MAP'].mean() > 65.0) & (relevant_rows['vasoactive/inotropic'] == 0.0).all() & (relevant_rows['Lactate'] < 2).any()
            circ_cond = ((relevant_rows['MAP'].mean() <= 65.0) | (relevant_rows['vasoactive/inotropic'] == 1.0).any()) & (relevant_rows['Lactate'] >= 2).any()
            
            # Annotation 결정
            if no_circ_cond:
                targ.at[idx, 'Annotation'] = 'no_circ'
            elif circ_cond:
                targ.at[idx, 'Annotation'] = 'circ'
            else:
                targ.at[idx, 'Annotation'] = 'ambiguous'

    return targ


def optimized_septicshock_labeler(df):
    targ = df.copy()
    targ['Shock_next_12h'] = np.nan
  
    for stay_id in tqdm(targ['patientunitstayid'].unique()):
        stay_df = targ[targ['patientunitstayid'] == stay_id].sort_values(by='Time_since_ICU_admission')
        stay_df['endpoint_window'] = stay_df['Time_since_ICU_admission'] + 12

        for idx, row in stay_df.iterrows():
            current_time = row['Time_since_ICU_admission']
            endpoint_window = row['endpoint_window']

       
            future_rows = stay_df[(stay_df['Time_since_ICU_admission'] > current_time) & (stay_df['Time_since_ICU_admission'] <= endpoint_window)]

            if any(future_rows['Annotation'] == 'circ'):
                targ.loc[idx, 'Shock_next_12h'] = 1
            else:
                targ.loc[idx, 'Shock_next_12h'] = 0

    return targ

after_annoted = annotation(after_sofa)
print('Annotation:\n', after_annoted['Annotation'].value_counts())

target = after_annoted.reset_index(drop=True)
final = optimized_septicshock_labeler(target)

100%|██████████| 14236/14236 [20:45<00:00, 11.43it/s]


Annotation:
 no_circ      1575878
ambiguous     778948
circ          151377
Name: Annotation, dtype: int64


100%|██████████| 14236/14236 [34:43<00:00,  6.83it/s] 


In [241]:
# 클래스 갯수 확인 24h
print('num case1', len(final[(final['Annotation']=='no_circ')&(final['Shock_next_12h']==0)]))
print('num case2', len(final[(final['Annotation']=='no_circ')&(final['Shock_next_12h']==1)]))
print('num case3', len(final[(final['Annotation']=='circ')&(final['Shock_next_12h']==0)]))
print('num case4', len(final[(final['Annotation']=='circ')&(final['Shock_next_12h']==1)]))

num case1 1560092
num case2 15786
num case3 8033
num case4 143344


In [242]:
intake = pd.read_csv("/Users/DAHS/Desktop/early_prediction_of_circ_scl/eicu-crd/2.0/intakeOutput.csv.gz", compression='gzip')
med = intake[intake['cellpath'].str.contains('intake', case=False, regex=True)]

antibiotic = ['vanco3.96 mg/ml (ml/hr)',
'vanco0.40mg/ml (ml/hr)',
'vanco 0.04 mg/ml (ml/hr)',
'NS with antibiotics (ml/hr)',
'NSS with anti (ml/hr)',
'NS antibx (ml/hr)',
'antibiotics (ml/hr)',
'antibiotic (ml/hr)',
'antibiotics (ml/hr)',
'Volume (mL)-vancomycin 1,250 mg in sodium chloride 0.9% 250 mL IVPB',
'Volume (mL)-vancomycin 1,500 mg in sodium chloride 0.9% 250 mL IVPB',
'Volume (mL)-vancomycin (VANCOCIN) 750 mg in sodium chloride 0.9 % 250 mL IVPB',
'vanco',
'vancomycin',
'Volume (mL)-vancomycin (VANCOCIN) IVPB 750 mg/150 ml premix',
'Volume (mL)-vancomycin (VANCOCIN) IVPB 1000 mg/200 mL premix',
'Volume (mL)-vancomycin (VANCOCIN) 2,000 mg in sodium chloride 0.9 % 500 mL IVPB',
'Volume (mL)-vancomycin (VANCOCIN) 1,250 mg in sodium chloride 0.9 % 250 mL IVPB',
'Volume (mL)-vancomycin (VANCOCIN) 500 mg in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-vancomycin (VANCOCIN) 1,500 mg in sodium chloride 0.9 % 500 mL IVPB',
'Volume (mL)-vancomycin (VANCOCIN) 750 mg in sodium chloride 0.9 % 150 mL IVPB',
'Volume (mL)-vancomycin (VANCOCIN) 1,500 mg in sodium chloride 0.9 % 250 mL IVPB',
'Volume (mL)-vancomycin (VANCOCIN) 1,750 mg in sodium chloride 0.9 % 500 mL IVPB',
'Volume (mL)-vancomycin in dextrose premix 1 g',
'Volume (mL)-vancomycin (VANCOCIN) 1,500 mg in dextrose 5 % 250 mL IVPB',
'Volume (mL)-vancomycin (VANCOCIN) 1,750 mg in dextrose 5 % 500 mL IVPB',
'Volume (mL)-vancomycin in dextrose premix 1,000 mg',
'Volume (mL)-vancomycin (VANCOCIN) 1,250 mg in dextrose 5 % 250 mL IVPB',
'Volume (mL)-vancomycin (VANCOCIN) 750 mg in dextrose 5 % 250 mL IVPB',
'Volume (mL)-meropenem (MERREM) 500 mg in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-meropenem (MERREM) 1 g in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-meropenem (MERREM) 2 g in sodium chloride 0.9 % 100 mL IVPB',
'meropenem',
'Volume (mL)-imipenem-cilastatin (PRIMAXIN) 250 mg in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-imipenem-cilastatin (PRIMAXIN) 500 mg in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 3.375 g in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 4.5 g in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) in dextrose 5% premix IVPB 3.375 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) IVPB 3.375 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) IVPB 2.25 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) IVPB 4.5 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) FOUR HOUR EXTENDED INFUSION IVPB 3.375 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 3.375 g in sodium chloride 0.9 % 100 mL thirty minute IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 3.375 g in sodium chloride 0.9 % 100 mL FOUR HOUR EXTENDED INFUSION IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 3.375 g in sodium chloride 0.9 % 100 mL EXTENDED INFUSION IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 2.25 g in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) thirty minute IVPB 4.5 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 4.5 g in sodium chloride 0.9 % 100 mL thirty minute IVPB',
'Volume (mL)-tigecycline (TYGACIL) 50 mg in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-ciprofloxacin (CIPRO) 400 mg in dextrose 5% 200 mL IVPB',
'Volume (mL)-ciprofloxacin (CIPRO) IVPB 400 mg',
'Volume (mL)-clindamycin 900 mg in sodium chloride 0.9% 50 mL IVPB',
'Volume (mL)-clindamycin 600 mg in sodium chloride 0.9% 50 mL IVPB',
'Volume (mL)-clindamycin (CLEOCIN) IVPB 900 mg',
'Volume (mL)-clindamycin (CLEOCIN) IVPB 600 mg',
'Volume (mL)-doxycycline (VIBRAMYCIN) 100 mg in sodium chloride 0.9 % 250 mL IVPB',
'Volume (mL)-doxycycline (VIBRAMYCIN) 100 mg in dextrose 5 % 250 mL IVPB',
'Volume (mL)-ampicillin-sulbactam (UNASYN) 3 g in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-ampicillin (OMNIPEN) 2,000 mg in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-ampicillin-sulbactam (UNASYN) 1.5 g in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-ampicillin (OMNIPEN) 2 g in sodium chloride 0.9 % 50 mL IVPB',
'Volume (mL)-ampicillin-sulbactam (UNASYN) 1.5 g in sodium chloride 0.9 % 50 mL IVPB',
'Volume (mL)-ampicillin (OMNIPEN) 12 g in sodium chloride 0.9 % 500 mL IVPB',
'Volume (mL)-ampicillin (OMNIPEN) 2 g in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-azithromycin (ZITHROMAX) 500 mg in sodium chloride 0.9 % 250 mL IVPB',
'Volume (mL)-azithromycin (ZITHROMAX) 500 mg in dextrose 5 % 250 mL IVPB',
'Volume (mL)-ciprofloxacin (CIPRO) 400 mg in dextrose 5% 200 mL IVPB',
'Volume (mL)-ciprofloxacin (CIPRO) IVPB 400 mg',
'Volume (mL)-erythromycin (ERYTHROCIN) 250 mg in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-fluconazole (DIFLUCAN) IVPB 200 mg',
'Volume (mL)-fluconazole (DIFLUCAN) IVPB 400 mg',
'Volume (mL)-fluconazole (DIFLUCAN) IVPB 100 mg',
'Volume (mL)-levofloxacin (LEVAQUIN) in dextrose 5% premix IVPB 750 mg',
'Volume (mL)-levofloxacin (LEVAQUIN) in dextrose 5% premix IVPB 500 mg',
'Volume (mL)-levofloxacin (LEVAQUIN) IVPB 500 mg',
'Volume (mL)-levofloxacin (LEVAQUIN) IVPB 750 mg',
'Volume (mL)-linezolid (ZYVOX) IVPB 600 mg',
'Volume (mL)-meropenem (MERREM) 500 mg in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-meropenem (MERREM) 1 g in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-meropenem (MERREM) 2 g in sodium chloride 0.9 % 100 mL IVPB',
'meropenem',
'Volume (mL)-nafcillin (NAFCIL) 2,000 mg in sodium chloride 0.9 % 100 mL IVPB',
'nafcillin 500mg/hr',
'Volume (mL)-penicillin G potassium 2.5 Million Units in dextrose 5 % 100 mL IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 3.375 g in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 4.5 g in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) in dextrose 5% premix IVPB 3.375 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) IVPB 3.375 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) IVPB 2.25 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) IVPB 4.5 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) FOUR HOUR EXTENDED INFUSION IVPB 3.375 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 3.375 g in sodium chloride 0.9 % 100 mL thirty minute IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 3.375 g in sodium chloride 0.9 % 100 mL FOUR HOUR EXTENDED INFUSION IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 3.375 g in sodium chloride 0.9 % 100 mL EXTENDED INFUSION IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 2.25 g in sodium chloride 0.9 % 100 mL IVPB',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) thirty minute IVPB 4.5 g',
'Volume (mL)-piperacillin-tazobactam (ZOSYN) 4.5 g in sodium chloride 0.9 % 100 mL thirty minute IVPB',
'Volume (mL)-tigecycline (TYGACIL) 50 mg in sodium chloride 0.9 % 100 mL IVPB']

antibio = med[med['celllabel'].isin(antibiotic)][['patientunitstayid', 'intakeoutputoffset']]
antibio['intakeoutputoffset'] = np.rint(antibio['intakeoutputoffset']/60)

In [243]:
antibio['Anotibio_add'] = 1
antibio = antibio.rename(columns={'intakeoutputoffset':'Time_since_ICU_admission'})

In [244]:
add_antibio = pd.merge(final, antibio, how = 'left', on = ['patientunitstayid', 'Time_since_ICU_admission'])
add_antibio = add_antibio.fillna(0)

In [245]:
add_antibio['Antibiotics'] = add_antibio[['Anotibio_add', 'Antibiotics']].sum(axis=1)
add_antibio = add_antibio.drop('Anotibio_add', axis = 1)
add_antibio['Antibiotics'] = add_antibio['Antibiotics'].apply(lambda x: 1 if not pd.isna(x) and x > 0 else 0)

In [247]:
add_antibio.to_csv('eICU-CIRC(12h).csv.gz')